In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.2.16'

## Create an engine - interface to the database

In [3]:
from sqlalchemy import create_engine

In [4]:
engine = create_engine('sqlite:///:memory:', echo=True)

## Create Base - base class holding all the mappings between the Python code and database tables

In [5]:
from sqlalchemy.ext.declarative import declarative_base

In [6]:
Base = declarative_base()

## Create a User entity

In [7]:
from sqlalchemy import Column, Integer, String

In [8]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    password = Column(String(12))
    
    def __repr__(self):
        return f'<User(name={self.name}, fullname={self.fullname}, password={self.password})>'

In [9]:
Base.metadata.create_all(engine)

2019-01-21 13:57:24,177 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-21 13:57:24,178 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:57:24,179 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-21 13:57:24,180 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:57:24,181 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-01-21 13:57:24,181 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:57:24,184 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(50), 
	fullname VARCHAR(50), 
	password VARCHAR(12), 
	PRIMARY KEY (id)
)


2019-01-21 13:57:24,184 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:57:24,185 INFO sqlalchemy.engine.base.Engine COMMIT


In [10]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [11]:
ed_user.name

'ed'

In [12]:
str(ed_user.id)

'None'

## Create a session object (not instantiated)

In [13]:
from sqlalchemy.orm import sessionmaker

In [14]:
Session = sessionmaker(bind=engine) # this adds the interface to the database straight away

but we can also use
```Session = sessionmaker()```
and then add the engine later with:
```Session.configure(bind=engine)```
this is useful for using in app factory where we do not want to bind it initially.


## Create a session instance

In [15]:
session = Session()

## Add object to database

In [16]:
session.add(ed_user)

Now the ed_user is **pending** - its inside a transaction in the session waiting for commit() to be saved to the database.

## Retrieving pending object will flush it giving it a unique ID

In [17]:
our_user = session.query(User).filter_by(name='ed').first()

2019-01-21 13:57:24,259 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-21 13:57:24,260 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-21 13:57:24,261 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')
2019-01-21 13:57:24,263 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-01-21 13:57:24,263 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


## The object retrieved from the database is the same object that was used to create it!

In [18]:
our_user is ed_user

True

## Add multiple objects with add_all()

In [19]:
session.add_all(
    [
        User(name='wendy', fullname='Wendy Williams', password='foobar'),
        User(name='mary', fullname='Mary Contrary', password='xfsdgfsd'),
        User(name='fred', fullname='Fred Flinstone', password='club'),
    ]
)

## Change existing attribute

In [20]:
ed_user.password = 'newpass'

## Now the changes to already existing objects are in ```session.dirty```

In [21]:
session.dirty

IdentitySet([<User(name=ed, fullname=Ed Jones, password=newpass)>])

## And new items that are pending are in ```session.new```

In [22]:
session.new

IdentitySet([<User(name=wendy, fullname=Wendy Williams, password=foobar)>, <User(name=mary, fullname=Mary Contrary, password=xfsdgfsd)>, <User(name=fred, fullname=Fred Flinstone, password=club)>])

## To save all pending changes we issue ```session.commit()``` command

In [23]:
session.commit()

2019-01-21 13:57:24,320 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2019-01-21 13:57:24,320 INFO sqlalchemy.engine.base.Engine ('newpass', 1)
2019-01-21 13:57:24,321 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-21 13:57:24,322 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2019-01-21 13:57:24,322 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-21 13:57:24,323 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xfsdgfsd')
2019-01-21 13:57:24,323 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-21 13:57:24,324 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'club')
2019-01-21 13:57:24,324 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
ed_user.id

2019-01-21 13:57:24,330 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-21 13:57:24,331 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2019-01-21 13:57:24,331 INFO sqlalchemy.engine.base.Engine (1,)


1

In [25]:
ed_user.password

'newpass'

## How to rollback the changes that are pending (not commited yet).

In [26]:
ed_user.name = 'Eduardo'

In [27]:
fake_user = User(name='fakeuser', fullname='Invalid', password='fsdfsd')

In [28]:
session.add(fake_user)

In [29]:
fake_user in session

True


## Query the session to flush the pending changes

In [30]:
session.query(User).filter(User.name.in_(['Eduardo', 'fakeuser'])).all()

2019-01-21 13:57:24,387 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2019-01-21 13:57:24,388 INFO sqlalchemy.engine.base.Engine ('Eduardo', 1)
2019-01-21 13:57:24,389 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-21 13:57:24,389 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', 'fsdfsd')
2019-01-21 13:57:24,390 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2019-01-21 13:57:24,391 INFO sqlalchemy.engine.base.Engine ('Eduardo', 'fakeuser')


[<User(name=Eduardo, fullname=Ed Jones, password=newpass)>,
 <User(name=fakeuser, fullname=Invalid, password=fsdfsd)>]

## Now rollback to revert the changes

In [31]:
session.rollback()

2019-01-21 13:57:24,397 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [32]:
ed_user.name # the name is again 'ed'

2019-01-21 13:57:24,407 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-21 13:57:24,408 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2019-01-21 13:57:24,408 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [33]:
fake_user in session # and the fake_user got removed from the session transaction

False

# Querying the database 

In [34]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2019-01-21 13:57:24,427 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2019-01-21 13:57:24,427 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


## We can also use ORM descriptors as arguments to query - then columns get pulled out 

In [35]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2019-01-21 13:57:24,437 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-01-21 13:57:24,437 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


## Query can be limited and offset as well using slicing

In [36]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2019-01-21 13:57:24,447 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2019-01-21 13:57:24,448 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name=wendy, fullname=Wendy Williams, password=foobar)>
<User(name=mary, fullname=Mary Contrary, password=xfsdgfsd)>


## Filtering by keyword arguments is done with ```filter_by()```

In [37]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'): 
    # This always return a tuple so we unpack it - name,
    print(name)

2019-01-21 13:57:24,457 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-01-21 13:57:24,457 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


## For filtering using SQL expressions we use ```filter()```

In [38]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2019-01-21 13:57:24,466 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-01-21 13:57:24,467 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


## The result of a query can be further filtered, we can chain operations

In [39]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2019-01-21 13:57:24,478 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2019-01-21 13:57:24,479 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name=ed, fullname=Ed Jones, password=newpass)>


## filter() accepts operators:
- equals -  ==
- not equals - !=
- LIKE - query.filter(User.name.like('%ed%')) - case sensitive like
- ILIKE - query.filter(User.name.ilike('%ed%)) - case insensitive like
- IN - query.filter(User.name.in_(['ed', 'wendy']))
- NOT IN - (Use ~ to negate) query.filter(~User.name.in_(['ed', 'wendy']))
- IS NULL - query.filter(User.name == None) or more pythonic: query.filter(User.name.is_(None))
- IS NOT NULL - query.filtr(USer.name != None) or more pythonic: query.filter(User.name.isnot(None))
- AND - query.filter(and_(User.name=='ed', User.fullname='Ed Jones'))
    we can also just add more expressions to achieve AND operator
    query.filter(User.name=='ed', User.fullname='Ed Jones')
    or chain filters themselves:
    query.filter(User.name=='ed').filter(User.fullname='Ed Jones')
- OR - query.filter(or_(User.name=='ed', User.name=='wendy))
- MATCH - query.filter(User.name.match('wendy')

## Returning lists and scalars

In [40]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)

In [41]:
query.all() # Returns a list of results

2019-01-21 13:57:24,499 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-01-21 13:57:24,499 INFO sqlalchemy.engine.base.Engine ('%ed',)


[<User(name=ed, fullname=Ed Jones, password=newpass)>,
 <User(name=fred, fullname=Fred Flinstone, password=club)>]

In [42]:
query.first() # Returns the first result in the list of results

2019-01-21 13:57:24,509 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2019-01-21 13:57:24,510 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


<User(name=ed, fullname=Ed Jones, password=newpass)>

In [43]:
# user = query.one() # Throws an error if multiple or no rows found.

## Use ```one_or_none()``` to get ```None``` if no results found (throws error if multiple)

In [44]:
query = session.query(User.id).filter(User.name=='ed').order_by(User.id)

## ```scalar()``` uses ```one()``` on query and returns only the first column in the row

In [45]:
query.scalar() # will return the user.id column only

2019-01-21 13:57:24,543 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id 
FROM users 
WHERE users.name = ? ORDER BY users.id
2019-01-21 13:57:24,544 INFO sqlalchemy.engine.base.Engine ('ed',)


1

# Using textual SQL 

## use ```text()``` in filter() and order_by()

In [46]:
from sqlalchemy import text

In [47]:
for user in session.query(User).filter(text('id<224')).order_by(text('id')).all():
    print(user.name)

2019-01-21 13:57:24,562 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<224 ORDER BY id
2019-01-21 13:57:24,563 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


## Parameters can be specified using string based SQL and a colon - to specify values we use ```params()```

In [48]:
session.query(User).filter(text('id<:value and name=:name')).params(value=224, name='fred').order_by(User.id).one()

2019-01-21 13:57:24,572 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<? and name=? ORDER BY users.id
2019-01-21 13:57:24,572 INFO sqlalchemy.engine.base.Engine (224, 'fred')


<User(name=fred, fullname=Fred Flinstone, password=club)>

## We can also use raw SQL - use ```from_statement()```

In [49]:
session.query(User).from_statement(text('SELECT * FROM users where name=:name')).params(name='ed').all()

2019-01-21 13:57:24,601 INFO sqlalchemy.engine.base.Engine SELECT * FROM users where name=?
2019-01-21 13:57:24,602 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User(name=ed, fullname=Ed Jones, password=newpass)>]

## We can also map columns positionally 

In [50]:
stmt = text('SELECT name, id, fullname, password FROM users where name=:name')

### Now pass columns mapping to name, id, fullname, password 

In [51]:
stmt = stmt.columns(User.name, User.id, User.fullname, User.password)

In [52]:
session.query(User).from_statement(stmt).params(name='ed').all()

2019-01-21 13:57:24,634 INFO sqlalchemy.engine.base.Engine SELECT name, id, fullname, password FROM users where name=?
2019-01-21 13:57:24,635 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User(name=ed, fullname=Ed Jones, password=newpass)>]

## Counting - use ```count()```

In [53]:
session.query(User).filter(User.name.like('%ed')).count()

2019-01-21 13:57:24,647 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?) AS anon_1
2019-01-21 13:57:24,650 INFO sqlalchemy.engine.base.Engine ('%ed',)


2

## We can also specify what should be counted using ```func```

In [54]:
from sqlalchemy import func

In [55]:
session.query(func.count(User.name), User.name).group_by(User.name).all()

2019-01-21 13:57:24,669 INFO sqlalchemy.engine.base.Engine SELECT count(users.name) AS count_1, users.name AS users_name 
FROM users GROUP BY users.name
2019-01-21 13:57:24,670 INFO sqlalchemy.engine.base.Engine ()


[(1, 'ed'), (1, 'fred'), (1, 'mary'), (1, 'wendy')]

### this counts how many 'ed' are there, how many 'fred' etc..

# Adding relationships 

In [56]:
from sqlalchemy import ForeignKey

In [57]:
from sqlalchemy.orm import relationship

In [58]:
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('User', back_populates='addresses')
    
    def __repr__(self):
        return f'<Address(email_address={self.email_address})>'

In [59]:
Base.metadata.create_all(engine)

2019-01-21 13:57:24,711 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-01-21 13:57:24,712 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:57:24,713 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-01-21 13:57:24,714 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:57:24,714 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-01-21 13:57:24,715 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:57:24,717 INFO sqlalchemy.engine.base.Engine COMMIT


### the user_id and user link Address to a User of whom this is the email address

## We also have to add the relationship on User to achieve one to many relationship

In [60]:
User.addresses = relationship('Address', order_by=Address.id, back_populates='user')

## Add related objects

In [61]:
jack = User(name='jack', fullname='Jack Bean', password='dsdfsdf')

In [62]:
jack.addresses

[]

## We can add new addresses to user jack as a list of Address objects

In [63]:
jack.addresses = [
    Address(email_address='jack@google.com'),
    Address(email_address='j25@yahoo.com')
]

In [64]:
jack.addresses

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

##  Because the relation is bidirectional from User to Address and from Address to User addresses added in user automatically give access to user in those addresses.

In [65]:
jack.addresses[1].user # we are accessing Address.user relationship

<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>

## If we now add jack to the session the new Addresses also will get added - cascading

In [66]:
session.add(jack)

In [67]:
session.commit()

2019-01-21 13:57:24,798 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-21 13:57:24,799 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'dsdfsdf')
2019-01-21 13:57:24,801 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-01-21 13:57:24,802 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 5)
2019-01-21 13:57:24,803 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-01-21 13:57:24,803 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 5)
2019-01-21 13:57:24,804 INFO sqlalchemy.engine.base.Engine COMMIT


# Note that the values in the relationship are queried !!! They are not preloaded - if we need them in one go with the parent object we must use joins.

In [68]:
jack.addresses

2019-01-21 13:57:24,811 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-21 13:57:24,812 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2019-01-21 13:57:24,813 INFO sqlalchemy.engine.base.Engine (5,)
2019-01-21 13:57:24,816 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2019-01-21 13:57:24,817 INFO sqlalchemy.engine.base.Engine (5,)


[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

# QUERYING MORE THAN ONE TABLE - RELATIONSHIPS AND JOINS

## To create an implicit join we can just add more entities to query

In [69]:
for u, a in session.query(User, Address).filter(User.id==Address.user_id).filter(Address.email_address=='jack@google.com').all():
    print(u)
    print(a)

2019-01-21 13:57:24,826 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id AND addresses.email_address = ?
2019-01-21 13:57:24,827 INFO sqlalchemy.engine.base.Engine ('jack@google.com',)
<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>
<Address(email_address=jack@google.com)>


## Joins are used to link 2 tables together

Join table User to Address

In [70]:
session.query(User).join(Address).filter(Address.email_address=='jack@google.com').all()

2019-01-21 13:57:24,838 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users JOIN addresses ON users.id = addresses.user_id 
WHERE addresses.email_address = ?
2019-01-21 13:57:24,839 INFO sqlalchemy.engine.base.Engine ('jack@google.com',)


[<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>]

## If the are no foreign keys or several we have to be more explicit in defining the join

In [71]:
query = session.query(User)

In [72]:
query.join(Address, User.id==Address.user_id).all() 
# explicitly say we need rows from addresses table where the user ids match

2019-01-21 13:57:24,859 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-01-21 13:57:24,860 INFO sqlalchemy.engine.base.Engine ()


[<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>]

In [73]:
query.join(User.addresses).all() # specifies relationship from left to right User -> Email Addresses

2019-01-21 13:57:24,869 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-01-21 13:57:24,870 INFO sqlalchemy.engine.base.Engine ()


[<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>]

In [74]:
query.join(Address, User.addresses).all() # As above but shows the target of the join (Address)

2019-01-21 13:57:24,880 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-01-21 13:57:24,881 INFO sqlalchemy.engine.base.Engine ()


[<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>]

In [75]:
query.join('addresses').all() # As above but giving the relation as a string

2019-01-21 13:57:24,896 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-01-21 13:57:24,897 INFO sqlalchemy.engine.base.Engine ()


[<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>]

##  Left out join is defined the same

In [76]:
query.outerjoin(User.addresses).all()

2019-01-21 13:57:24,908 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users LEFT OUTER JOIN addresses ON users.id = addresses.user_id
2019-01-21 13:57:24,909 INFO sqlalchemy.engine.base.Engine ()


[<User(name=ed, fullname=Ed Jones, password=newpass)>,
 <User(name=wendy, fullname=Wendy Williams, password=foobar)>,
 <User(name=mary, fullname=Mary Contrary, password=xfsdgfsd)>,
 <User(name=fred, fullname=Fred Flinstone, password=club)>,
 <User(name=jack, fullname=Jack Bean, password=dsdfsdf)>]

## When querying using same table more than once we my need to use aliases to it

In [77]:
from sqlalchemy.orm import aliased

In [78]:
adalias1 = aliased(Address)

In [79]:
adalias2 = aliased(Address)

In [80]:
for username, email1, email2 in (
    session.query(User.name, adalias1.email_address, adalias2.email_address)
    .join(adalias1, User.addresses)
    .join(adalias2, User.addresses)
    .filter(adalias1.email_address=='jack@google.com')
    .filter(adalias2.email_address=='j25@yahoo.com')):
    print(username, email1, email2)
    

2019-01-21 13:57:24,948 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, addresses_1.email_address AS addresses_1_email_address, addresses_2.email_address AS addresses_2_email_address 
FROM users JOIN addresses AS addresses_1 ON users.id = addresses_1.user_id JOIN addresses AS addresses_2 ON users.id = addresses_2.user_id 
WHERE addresses_1.email_address = ? AND addresses_2.email_address = ?
2019-01-21 13:57:24,949 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')
jack jack@google.com j25@yahoo.com


## Using subqeries

We want to load User object with a count of their email addresses. First we can get the counts groups by user ids and then join the to the parent - the user. To get the counts we use a subquery with func.

In [81]:
stmt = (
    session.query(Address.user_id, func.count('*').label('address_count'))
    .group_by(Address.user_id)
    .subquery()
)

###  The ```subquery()``` produces a SELECT statement embedded within an alias
The stmt with a subquery behaves like a new Table and columns are accessed using ```stmt.c```

In [82]:
for u, count in (
    session.query(User, stmt.c.address_count)
    .outerjoin(stmt, User.id==stmt.c.user_id)
    .order_by(User.id)):
    print(u, count)

2019-01-21 13:57:24,969 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, anon_1.address_count AS anon_1_address_count 
FROM users LEFT OUTER JOIN (SELECT addresses.user_id AS user_id, count(?) AS address_count 
FROM addresses GROUP BY addresses.user_id) AS anon_1 ON users.id = anon_1.user_id ORDER BY users.id
2019-01-21 13:57:24,969 INFO sqlalchemy.engine.base.Engine ('*',)
<User(name=ed, fullname=Ed Jones, password=newpass)> None
<User(name=wendy, fullname=Wendy Williams, password=foobar)> None
<User(name=mary, fullname=Mary Contrary, password=xfsdgfsd)> None
<User(name=fred, fullname=Fred Flinstone, password=club)> None
<User(name=jack, fullname=Jack Bean, password=dsdfsdf)> 2


## Selecting entities from subqueries (not only columns with .c.<column_name>)

In [83]:
stmt = session.query(Address).filter(Address.email_address != 'j25@yahoo.com').subquery()

### We have to make an alias out of the subquery using ```aliased()``` 

In [84]:
adalias = aliased(Address, stmt)

In [85]:
for user, address in session.query(User, adalias).join(adalias, User.addresses):
    print(user)
    print(address)

2019-01-21 13:57:25,012 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, anon_1.id AS anon_1_id, anon_1.email_address AS anon_1_email_address, anon_1.user_id AS anon_1_user_id 
FROM users JOIN (SELECT addresses.id AS id, addresses.email_address AS email_address, addresses.user_id AS user_id 
FROM addresses 
WHERE addresses.email_address != ?) AS anon_1 ON users.id = anon_1.user_id
2019-01-21 13:57:25,013 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com',)
<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>
<Address(email_address=jack@google.com)>


## Use ```exists()``` to confirm query returns any rows - returns a boolean

In [86]:
from sqlalchemy.sql import exists

In [87]:
stmt = exists().where(Address.user_id==User.id)

In [88]:
for name, in session.query(User.name).filter(stmt):
    print(name)

2019-01-21 13:57:25,042 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE EXISTS (SELECT * 
FROM addresses 
WHERE addresses.user_id = users.id)
2019-01-21 13:57:25,043 INFO sqlalchemy.engine.base.Engine ()
jack


# Eager loading - use to eliminate number of queries when selecting related objects - as normally by default, relations are loaded lazily (only when accessed they emit a new query which can be really bad)

## We invoke different loading via function in query options().

## ```selectinload()```

### to load email addresses per user we can use ```selectinload``` - they will get added to the rows

In [89]:
from sqlalchemy.orm import selectinload

In [90]:
jack = session.query(User).options(selectinload(User.addresses)).filter_by(name='jack').one()

2019-01-21 13:57:25,062 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2019-01-21 13:57:25,062 INFO sqlalchemy.engine.base.Engine ('jack',)
2019-01-21 13:57:25,065 INFO sqlalchemy.engine.base.Engine SELECT users_1.id AS users_1_id, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users AS users_1 JOIN addresses ON users_1.id = addresses.user_id 
WHERE users_1.id IN (?) ORDER BY users_1.id, addresses.id
2019-01-21 13:57:25,066 INFO sqlalchemy.engine.base.Engine (5,)


In [91]:
jack

<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>

In [92]:
jack.addresses # Notice that it does not emit a new query - addresses are now loaded

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

## ```joinedload()```

Emits a **left outer** join by default. So the left, parent object and its children are loaded in one step.

In [93]:
from sqlalchemy.orm import joinedload

In [94]:
jack = session.query(User).options(joinedload(User.addresses)).filter_by(name='jack').one()

2019-01-21 13:57:25,108 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses_1.id AS addresses_1_id, addresses_1.email_address AS addresses_1_email_address, addresses_1.user_id AS addresses_1_user_id 
FROM users LEFT OUTER JOIN addresses AS addresses_1 ON users.id = addresses_1.user_id 
WHERE users.name = ? ORDER BY addresses_1.id
2019-01-21 13:57:25,109 INFO sqlalchemy.engine.base.Engine ('jack',)


## The join created in joinedload is anonymously aliased and does not affect the main query - this is the difference from normal ```join()```. We use it to optimize loading of the relations.

## use normal ```join()``` and ```contains_eager()``` to mark which entities to load eagerly

In [95]:
from sqlalchemy.orm import contains_eager

In [96]:
jacks_addresses = (
    session.query(Address)
    .join(Address.user)
    .filter(User.name=='jack')
    .options(contains_eager(Address.user))
    .all()
)

2019-01-21 13:57:25,128 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses JOIN users ON users.id = addresses.user_id 
WHERE users.name = ?
2019-01-21 13:57:25,129 INFO sqlalchemy.engine.base.Engine ('jack',)


In [97]:
jacks_addresses

[<Address(email_address=jack@google.com)>,
 <Address(email_address=j25@yahoo.com)>]

In [98]:
jacks_addresses[0].user # already loaded

<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>

# Deleting objects 

In [99]:
session.delete(jack)

In [100]:
session.query(User).filter_by(name='jack').count() # jack object got deleted but the related objects not

2019-01-21 13:57:25,172 INFO sqlalchemy.engine.base.Engine UPDATE addresses SET user_id=? WHERE addresses.id = ?
2019-01-21 13:57:25,173 INFO sqlalchemy.engine.base.Engine ((None, 1), (None, 2))
2019-01-21 13:57:25,175 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id = ?
2019-01-21 13:57:25,176 INFO sqlalchemy.engine.base.Engine (5,)
2019-01-21 13:57:25,178 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?) AS anon_1
2019-01-21 13:57:25,179 INFO sqlalchemy.engine.base.Engine ('jack',)


0

In [101]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

2019-01-21 13:57:25,190 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2019-01-21 13:57:25,192 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')


2

## As we can see the jack got deleted but his related email addresses stay - to delete them together with their parent object we need to specify CASCADE

# Configure CASCADE for email address relation on User - we have to redo the session and make a new User.

In [102]:
session.close() # this will rollback the deletion of jack

2019-01-21 13:57:25,206 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [103]:
Base = declarative_base()

In [104]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    addresses = relationship('Address', back_populates='user', cascade='all, delete, delete-orphan')
    
    def __repr__(self):
        return f'<User(name={self.name}, fullname={self.fullname}, password={self.password})>'

In [105]:
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('User', back_populates='addresses')
    
    def __repr__(self):
        return f'<Address(email_address={self.email_address})>'

In [106]:
# load jack by his id = 5

In [107]:
jack = session.query(User).get(5)

2019-01-21 13:57:25,259 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-21 13:57:25,260 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2019-01-21 13:57:25,261 INFO sqlalchemy.engine.base.Engine (5,)


In [108]:
jack

<User(name=jack, fullname=Jack Bean, password=dsdfsdf)>

In [109]:
del jack.addresses[1] 
# remove one of his related emails - lazy load will get triggered as we did not preload addresses

2019-01-21 13:57:25,280 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id
2019-01-21 13:57:25,281 INFO sqlalchemy.engine.base.Engine (5,)


In [110]:
session.query(Address).filter(Address.user==jack).count() # Now only one address remains

2019-01-21 13:57:25,295 INFO sqlalchemy.engine.base.Engine DELETE FROM addresses WHERE addresses.id = ?
2019-01-21 13:57:25,296 INFO sqlalchemy.engine.base.Engine (2,)
2019-01-21 13:57:25,298 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id) AS anon_1
2019-01-21 13:57:25,299 INFO sqlalchemy.engine.base.Engine (5,)


1

## Now if we delete jack the other associated email will get deleted

In [111]:
session.delete(jack)

In [112]:
session.query(User).filter_by(name='jack').count()

2019-01-21 13:57:25,321 INFO sqlalchemy.engine.base.Engine DELETE FROM addresses WHERE addresses.id = ?
2019-01-21 13:57:25,322 INFO sqlalchemy.engine.base.Engine (1,)
2019-01-21 13:57:25,323 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id = ?
2019-01-21 13:57:25,323 INFO sqlalchemy.engine.base.Engine (5,)
2019-01-21 13:57:25,324 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?) AS anon_1
2019-01-21 13:57:25,325 INFO sqlalchemy.engine.base.Engine ('jack',)


0

In [113]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

2019-01-21 13:57:25,338 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2019-01-21 13:57:25,339 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')


0

# Many to Many relationships

## We will make a BlogPost which can have many Keyword(s).

In [114]:
from sqlalchemy import Table, Text

## We need an association table to link many blogs with many keywords

In [115]:
post_keywords = Table('post_keywords',
                      Base.metadata,
                      Column('post_id', ForeignKey('posts.id'), primary_key=True),
                      Column('keyword_id', ForeignKey('keywords.id'), primary_key=True))

In [116]:
class BlogPost(Base):
    __tablename__ = 'posts'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body = Column(Text)
    
    #many to many relationship to keyword
    keywords = relationship('Keyword', secondary=post_keywords, back_populates='posts')
    
    def __init__(self, headline, body, author):
        self.author = author
        self.headline = headline
        self.body = body
    
    def __repr__(self):
        return f'BlogPost<({self.headline}, {self.body}, {self.author})'

In [117]:
class Keyword(Base):
    __tablename__ = 'keywords'
    id = Column(Integer, primary_key=True)
    keyword = Column(String(50), nullable=False, unique=True)
    posts = relationship('BlogPost', secondary=post_keywords, back_populates='keywords')
    
    def __init__(self, keyword):
        self.keyword = keyword

## When using declarative we DO NOT have to use __init__ - all the attributes will be set for us automatically

## the ```secondary=``` parameter is the key parameter to make the many to many relationship - references the association table ```post_keywords``` that links BlogPost to Keyword

In [118]:
BlogPost.author = relationship(User, back_populates='posts') # add also author field

In [119]:
User.posts = relationship(BlogPost, back_populates='author', lazy='dynamic')

In [120]:
Base.metadata.create_all(engine)

2019-01-21 13:59:54,896 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-01-21 13:59:54,898 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:59:54,901 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-01-21 13:59:54,902 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:59:54,905 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("post_keywords")
2019-01-21 13:59:54,906 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:59:54,907 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("posts")
2019-01-21 13:59:54,908 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:59:54,909 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("keywords")
2019-01-21 13:59:54,909 INFO sqlalchemy.engine.base.Engine ()
2019-01-21 13:59:54,910 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE posts (
	id INTEGER NOT NULL, 
	user_id INTEGER, 
	headline VARCHAR(255) NOT NULL, 
	body TEXT, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-01

In [121]:
wendy = session.query(User).filter_by(name='wendy').one()

2019-01-21 14:00:40,080 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2019-01-21 14:00:40,081 INFO sqlalchemy.engine.base.Engine ('wendy',)


In [122]:
post = BlogPost("Wendy's Blog Post", "This is a test", wendy)

In [123]:
session.add(post)

In [124]:
post.keywords.append(Keyword('wendy'))
post.keywords.append(Keyword('firstpost'))

In [125]:
session.query(BlogPost).filter(BlogPost.keywords.any(keyword='firstpost')).all()

2019-01-21 14:03:38,044 INFO sqlalchemy.engine.base.Engine INSERT INTO keywords (keyword) VALUES (?)
2019-01-21 14:03:38,045 INFO sqlalchemy.engine.base.Engine ('wendy',)
2019-01-21 14:03:38,046 INFO sqlalchemy.engine.base.Engine INSERT INTO keywords (keyword) VALUES (?)
2019-01-21 14:03:38,047 INFO sqlalchemy.engine.base.Engine ('firstpost',)
2019-01-21 14:03:38,048 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (user_id, headline, body) VALUES (?, ?, ?)
2019-01-21 14:03:38,048 INFO sqlalchemy.engine.base.Engine (2, "Wendy's Blog Post", 'This is a test')
2019-01-21 14:03:38,049 INFO sqlalchemy.engine.base.Engine INSERT INTO post_keywords (post_id, keyword_id) VALUES (?, ?)
2019-01-21 14:03:38,049 INFO sqlalchemy.engine.base.Engine ((1, 1), (1, 2))
2019-01-21 14:03:38,052 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.user_id AS posts_user_id, posts.headline AS posts_headline, posts.body AS posts_body 
FROM posts 
WHERE EXISTS (SELECT 1 
FROM post_keywords,

[BlogPost<(Wendy's Blog Post, This is a test, <User(name=wendy, fullname=Wendy Williams, password=foobar)>)]

In [127]:
wendy.posts.filter(BlogPost.keywords.any(keyword='firstpost')). all()

2019-01-21 14:04:42,691 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.user_id AS posts_user_id, posts.headline AS posts_headline, posts.body AS posts_body 
FROM posts 
WHERE ? = posts.user_id AND (EXISTS (SELECT 1 
FROM post_keywords, keywords 
WHERE posts.id = post_keywords.post_id AND keywords.id = post_keywords.keyword_id AND keywords.keyword = ?))
2019-01-21 14:04:42,693 INFO sqlalchemy.engine.base.Engine (2, 'firstpost')


[BlogPost<(Wendy's Blog Post, This is a test, <User(name=wendy, fullname=Wendy Williams, password=foobar)>)]